<a href="https://colab.research.google.com/github/heffo42/flask-bio/blob/main/PUBMED_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import xml.etree.ElementTree as ET
import ftplib
import gzip

# Connect with FTP

In [2]:
ftp_server = 'ftp.ncbi.nlm.nih.gov'
ftp_session= ftplib.FTP(ftp_server)
ftp_session.login()
ftp_session.cwd('pubmed/baseline') #cwd to files

'250 CWD command successful'

In [3]:
all_files = ftp_session.nlst()
content_files = []
for c_file in all_files:
  if c_file[-3:] == '.gz':
    content_files.append(c_file) 
    ftp_session.retrbinary("RETR " + c_file, open(c_file, 'wb').write)

KeyboardInterrupt: ignored

# Processes Each File

In [4]:
content_files[0]

'pubmed21n0004.xml.gz'

In [5]:
filename = content_files[0]

In [6]:
input = gzip.open(filename, 'r')
tree = ET.parse(input)
root = tree.getroot()

## Process Each Article Within File

In [7]:
i = 10
for child in root:
  print(child)
  i -= 1
  if i == 0:
    break

<Element 'PubmedArticle' at 0x7fee63b62818>
<Element 'PubmedArticle' at 0x7fee63afd368>
<Element 'PubmedArticle' at 0x7fee63b09b38>
<Element 'PubmedArticle' at 0x7fee63b12f48>
<Element 'PubmedArticle' at 0x7fee63b28cc8>
<Element 'PubmedArticle' at 0x7fee63abddb8>
<Element 'PubmedArticle' at 0x7fee63ad1958>
<Element 'PubmedArticle' at 0x7fee63a6c7c8>
<Element 'PubmedArticle' at 0x7fee63a88048>
<Element 'PubmedArticle' at 0x7fee63a986d8>


In [14]:

def process_contributor(a, contrib_type):
  authors = []
  for author in a.findall(f'.//{contrib_type}'):
    current_author = {}
    l_name = author.find('./LastName')
    if l_name is not None:
      current_author['last'] = l_name.text
    f_name = author.find('./ForeName')
    if f_name is not None:
      current_author['fore'] = f_name.text
    authors.append(current_author)
  return authors


def processes_article(a):
  json_art = {}
  # print(ET.tostring(article, encoding='utf8').decode('utf8'))
  # a.findall('./MedlineCitation/DateCompleted/')

  # authors (1)
  json_art['authors'] = process_contributor(a, 'Author')

  # investigators (2)
  json_art['investigators'] = process_contributor(a, 'Investigator')


  # PubMed Identifier (3)
  json_art['pmid'] = a.find('.//PMID').text

  # accesing number NCT ID (4)

  # abstract (5)
  abstract = a.find('.//AbstractText')
  if abstract is not None:
    json_art['abstract'] = abstract.text

  # title (6)
  json_art['title'] = a.find('./MedlineCitation/Article/ArticleTitle').text

  # MeSH terms (7)

  # publication date (8)

 

  return json_art

  



In [16]:
processes_article(samples[0])

{'authors': [{'fore': 'D G', 'last': 'McDevitt'}],
 'investigators': [],
 'pmid': '92018',
 'title': 'Problems in the treatment of hypertension.'}

In [29]:
print(ET.tostring(samples[2], encoding='unicode'))

<PubmedArticle>
    <MedlineCitation Owner="NLM" Status="MEDLINE">
      <PMID Version="1">92020</PMID>
      <DateCompleted>
        <Year>1980</Year>
        <Month>01</Month>
        <Day>28</Day>
      </DateCompleted>
      <DateRevised>
        <Year>2005</Year>
        <Month>11</Month>
        <Day>17</Day>
      </DateRevised>
      <Article PubModel="Print">
        <Journal>
          <ISSN IssnType="Print">0552-2080</ISSN>
          <JournalIssue CitedMedium="Print">
            <Volume>24</Volume>
            <Issue>10</Issue>
            <PubDate>
              <Year>1979</Year>
              <Month>Oct</Month>
            </PubDate>
          </JournalIssue>
          <Title>Problemy gematologii i perelivaniia krovi</Title>
          <ISOAbbreviation>Probl Gematol Pereliv Krovi</ISOAbbreviation>
        </Journal>
        <ArticleTitle>[Problems in organizing the manufacture of antirhesus immunoglobulin].</ArticleTitle>
        <Pagination>
          <MedlinePgn>54-6</Me

In [30]:
for x in samples:
  q = x.findall('.//DataBankName[text()="ClinicalTrials.gov"]')
  if len(q) > 0:
    print(q)
    break





SyntaxError: ignored

In [ ]:
a = './/DataBankName[text()="ClinicalTrials.gov"]'

In [ ]:
x = processes_article(samples[9])

In [ ]:
x

{'abstract': "Development of a highly sensitive silver stain permits the characterization of trace cellular and body fluid proteins separated by the two-dimensional electrophoresis technique of O'Farrell. Many of the proteins detected by the silver stain in urine, spinal fluid, amniotic fluid, and cells were undetected with the widely used Coomassie blue stain. Trace polypeptides observed in Escherichia coli cell lysates with this silver stain could be detected previously only by growth in radioactive precursors followed by lengthy autoradiography. In situations that do not permit the use of radioactive labeling, as in human clinical studies, the enhanced ability to detect proteins achieved by the silver stain will facilitate metabolic studies and the screening for protein abnormalities in mutational studies and in genetic diseases.",
 'authors': [{'fore': 'C R', 'last': 'Merril'},
  {'fore': 'R C', 'last': 'Switzer'},
  {'fore': 'M L', 'last': 'Van Keuren'}],
 'investigators': [],
 'p

In [ ]:
if len(x['investigators']) > 0:
  print('woot')

In [13]:
sum(1 for _ in iter(root))

30000

In [10]:
samples = []
root_iter = iter(root)
for i in range(30):
  samples.append(next(root_iter))

In [ ]:
ET.tostring(a.findall('./MedlineCitation/Article/AuthorList/')[0])

b'<Author ValidYN="Y">\n            <LastName>Makar</LastName>\n            <ForeName>A B</ForeName>\n            <Initials>AB</Initials>\n          </Author>\n          '

# Sandbox

In [ ]:
a = next(iter(root))

In [ ]:
temp = iter(root)
next(temp)
a = next(temp)

In [ ]:
i = 0
j = 0
for a in root:
  j += 1
  if len():
    i += 1
    # print(a.find('.//AbstractText').text)
    # print(ET.tostring(a, encoding='utf8').decode('utf8'))

TypeError: ignored

AttributeError: ignored

In [ ]:
authors = []
for author in a.findall('.//Author'):
  current_author = {}
  l_name = author.find('./LastName')
  if l_name is not None:
    current_author['last'] = l_name.text
  f_name = author.find('./ForeName')
  if f_name is not None:
    current_author['fore'] = f_name.text
  authors.append(current_author)




In [ ]:
print(ET.tostring(a, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<PubmedArticle>
    <MedlineCitation Owner="NLM" Status="MEDLINE">
      <PMID Version="1">92019</PMID>
      <DateCompleted>
        <Year>1980</Year>
        <Month>01</Month>
        <Day>28</Day>
      </DateCompleted>
      <DateRevised>
        <Year>2005</Year>
        <Month>11</Month>
        <Day>17</Day>
      </DateRevised>
      <Article PubModel="Print">
        <Journal>
          <ISSN IssnType="Print">0552-2080</ISSN>
          <JournalIssue CitedMedium="Print">
            <Volume>24</Volume>
            <Issue>10</Issue>
            <PubDate>
              <Year>1979</Year>
              <Month>Oct</Month>
            </PubDate>
          </JournalIssue>
          <Title>Problemy gematologii i perelivaniia krovi</Title>
          <ISOAbbreviation>Probl Gematol Pereliv Krovi</ISOAbbreviation>
        </Journal>
        <ArticleTitle>[Different aspects of using antirhesus immunoglobulin].</ArticleTitle>
        <Pagination>
       